In [91]:
# import tensorflow
import tensorflow as tf
tf.__version__

'2.8.0'

In [92]:
import keras
keras.__version__

'2.8.0'

In [93]:
!pip install tflearn
# import libraries
from tflearn.datasets import oxflower17
from keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, Dropout, Activation, MaxPooling2D, Input
from keras.layers import BatchNormalization, Activation, Flatten, Dense


In [94]:
# load data
# X, Y = oxflower17.load_data(r'Data/jpg')
X,Y = oxflower17.load_data(one_hot=True)

In [95]:
X.shape, Y.shape
Y[0]  # data is already in categorical form

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [96]:
# make categorical
# Y=to_categorical(Y)
# Y.shape

In [97]:
from keras.layers.pooling import MaxPooling2D
# Define a VGG block:

def vgg_block(layer_in, n_filters, n_conv):
    # add conv layers
    for _ in range(n_conv):
        layer_in = Conv2D(n_filters, (3,3), padding='same',
                          activation='relu')(layer_in)

    layer_in = MaxPooling2D( (2,2), strides=(2,2))(layer_in)
    return layer_in

In [98]:
# build model

model=Sequential()
# 1st convolution layer stack and Maxppoling
input_layer = Input(shape=(224, 224,3))
# add vgg stack of layers
layer=vgg_block(input_layer, 64 , 2 )

# second convolutional layer
layer = vgg_block(layer, 128, 2)

# third convolutional layer
layer=vgg_block(layer, 256, 3)

# fourth convolutional layer
layer = vgg_block(layer, 512, 3)

# fith convolutional layer and maxpooling
layer = vgg_block(layer, 512, 3)

# 1st Dense layer
layer=Flatten()(layer)
layer=Dense(4096)(layer)

# Dropout
layer=(Dropout(0.5))(layer)
layer=(Activation('relu'))(layer)

# 2nd Dense layer
layer=Dense(4096)(layer)

# Droput 
layer=(Dropout(0.5))(layer)

# Normalization
layer=(BatchNormalization())(layer)
layer=(Activation)('relu')(layer)

# Ouput softmax layer
layer=(Dense(17))(layer)
layer=(Activation('softmax'))(layer)

model=Model(inputs=input_layer, outputs=layer)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_50 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_51 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_52 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_53 (Conv2D)          (None, 112, 112, 128)     147584    
                                                           

In [102]:
# compile model
model.compile(optimizer=tf.keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
    name='RMSprop',
)
, loss='categorical_crossentropy', metrics=['accuracy'])


# optimizer=keras.optimizers.RMSprop(learning_rate=2e-04


batch_size=20
history = model.fit(X,  Y, epochs=30,
                    batch_size=batch_size,
                    validation_split=0.2, verbose=1, shuffle=True)


Train on 1088 samples, validate on 272 samples
Epoch 1/30
1088/1088 [==============================] - ETA: 0s - loss: 3.5597 - acc: 0.0855

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


1088/1088 [==============================] - 55s 50ms/sample - loss: 3.5597 - acc: 0.0855 - val_loss: 39.3603 - val_acc: 0.0809
Epoch 2/30
1088/1088 [==============================] - 45s 41ms/sample - loss: 2.9972 - acc: 0.1526 - val_loss: 4.6815 - val_acc: 0.1507
Epoch 3/30
1088/1088 [==============================] - 44s 41ms/sample - loss: 2.8100 - acc: 0.1875 - val_loss: 6.0931 - val_acc: 0.1360
Epoch 4/30
1088/1088 [==============================] - 44s 41ms/sample - loss: 2.7095 - acc: 0.2077 - val_loss: 3.0244 - val_acc: 0.2206
Epoch 5/30
1088/1088 [==============================] - 45s 41ms/sample - loss: 2.6374 - acc: 0.2472 - val_loss: 2.9878 - val_acc: 0.1765
Epoch 6/30
1088/1088 [==============================] - 44s 41ms/sample - loss: 2.4623 - acc: 0.2794 - val_loss: 3.9704 - val_acc: 0.2978
Epoch 7/30
1088/1088 [==============================] - 45s 41ms/sample - loss: 2.3561 - acc: 0.3272 - val_loss: 2.3870 - val_acc: 0.3051
Epoch 8/30
1088/1088 [======================

In [ ]:
# Train model
model.fit(X, Y, batch_size=32, epochs=30, verbose=1, validation_split=0.2, shuffle=True)